In [1]:
#Funnel-Quality,Growth,Value method
#Nifty100 stocks pool
import yfinance as yf
import pandas as pd
import numpy as np


In [2]:
#Level 1: Quality
def fetch_quality_ratios(ticker):
    try:
        stock = yf.Ticker(ticker)
        
    
        # Fetching returnOnEquity
        returnOnEquity = pd.to_numeric(stock.info.get('returnOnEquity', 'N/A'), errors='coerce')
    
        # Fetching quickRatio
        quickRatio = pd.to_numeric(stock.info.get('quickRatio', 'N/A'), errors='coerce')
    
        # Fetching currentRatio
        currentRatio = pd.to_numeric(stock.info.get('currentRatio', 'N/A'), errors='coerce')
        
        # Fetching profitMargins
        profitMargins = pd.to_numeric(stock.info.get('profitMargins', 'N/A'), errors='coerce')
    
        #Fetching freecashflowyield
        freeCashflow = pd.to_numeric(stock.info.get('freeCashflow', 'N/A'), errors='coerce')
        sharesOutstanding = pd.to_numeric(stock.info.get('sharesOutstanding', 'N/A'), errors='coerce')
        freecashflowpershare = freeCashflow/sharesOutstanding
        previousClose= pd.to_numeric(stock.info.get('previousClose', 'N/A'), errors='coerce')
        freecashflowyield = freecashflowpershare/previousClose

    
     
        return {
            'Company': ticker,
            'returnOnEquity': returnOnEquity,
            'quickRatio': quickRatio,
            'currentRatio': currentRatio,
            'profitMargins': profitMargins,
            'freecashflowyield': freecashflowyield,
            
            
        }
    
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return {'Company': ticker}

companies = ["ABB.NS",
"ACC.NS",
"ADANIPORTS.NS",
"ADANIPOWER.NS",
"APOLLOHOSP.NS",
"ASIANPAINT.NS",
"ATUL.NS",
"AUROPHARMA.NS",
"AXISBANK.NS",
"BAJAJ-AUTO.NS",
"BAJAJFINSV.NS",
"BAJAJHLDNG.NS",
"BALKRISHNA.NS",
"BANDHANBNK.NS",
"BEL.NS",
"BHARATFORG.NS",
"BHARTIARTL.NS",
"BIOCON.NS",
"BOSCHLTD.NS",
"BPCL.NS",
"BRITANNIA.NS",
"CIPLA.NS",
"COALINDIA.NS",
"CONCOR.NS",
"DABUR.NS",
"DEEPAKNTR.NS",
"DIVISLAB.NS",
"DRREDDY.NS",
"ECLERX.NS",
"EICHERMOT.NS",
"EXIDEIND.NS",
"FEDERALBNK.NS",
"GODREJCP.NS",
"GRASIM.NS",
"HCLTECH.NS",
"HDFCBANK.NS",
"HDFCLIFE.NS",
"HEROMOTOCO.NS",
"HINDUNILVR.NS",
"HONAUT.NS",
"ICICIBANK.NS",
"IDFC.NS",
"IDEA.NS",
"INDIGO.NS",
"INFY.NS",
"ITC.NS",
"JINDALSTEL.NS",
"JSWSTEEL.NS",
"KOTAKBANK.NS",
"LUPIN.NS",
"M&M.NS",
"MARICO.NS",
"MARUTI.NS",
"METROPOLIS.NS",
"MSUMI.NS",
"MRF.NS",
"NESTLEIND.NS",
"NTPC.NS",
"OIL.NS",
"ONGC.NS",
"PAGEIND.NS",
"PEL.NS",
"PFC.NS",
"PIIND.NS",
"POWERGRID.NS",
"RAILTEL.NS",
"RELIANCE.NS",
"RELCAPITAL.NS",
"REPCOHOME.NS",
"SBIN.NS",
"SAIL.NS",
"SCHNEIDER.NS",
"SHRIRAMFIN.NS",
"SUNPHARMA.NS",
"TATACHEM.NS",
"TATACONSUM.NS",
"TATAMOTORS.NS",
"TATAPOWER.NS",
"TCS.NS",
"TECHM.NS",
"TITAN.NS",
"TVSMOTOR.NS",
"UJJIVAN.NS",
"ULTRACEMCO.NS",
"UPL.NS",
"VEDL.NS",
"WIPRO.NS",
"YESBANK.NS",
"ZEEL.NS"]
# Fetch financial ratios for each company
quality_ratios = [fetch_quality_ratios(ticker) for ticker in companies]

# Convert the list of dictionaries to a pandas DataFrame
result_df = pd.DataFrame(quality_ratios)

# Drop rows with missing values
result_df = result_df.dropna()
result_df

,Company,returnOnEquity,quickRatio,currentRatio,profitMargins,freecashflowyield
0,ABB.NS,0.22936,1.468,1.904,0.11556,0.011103
16,BHARTIARTL.NS,0.10416,0.214,0.482,0.05661,0.032440
27,DRREDDY.NS,0.21391,1.653,2.548,0.19243,0.021958
34,HCLTECH.NS,0.23944,2.375,2.580,0.14509,0.000520
44,INFY.NS,0.31604,1.696,1.986,0.15912,0.000279
56,NESTLEIND.NS,1.08023,0.373,0.948,0.15678,0.008830
78,TCS.NS,0.45524,2.142,2.437,0.18787,0.031376
86,WIPRO.NS,0.15270,2.302,2.429,0.12436,0.037342
88,ZEEL.NS,0.01088,1.061,4.298,-0.01182,0.010642


In [3]:
# Define whether higher or lower values are better for each ratio
higher_is_better = {
    'returnOnEquity': True,
    'quickRatio': True,
    'currentRatio': True,
    'profitMargins': True,
    'freecashflowyield': True,
}

# Calculate percentiles for each ratio
for ratio, is_higher_better in higher_is_better.items():
    result_df[f'{ratio} Percentile'] = result_df[ratio].rank(pct=True, ascending=is_higher_better) * 100

# Calculate Composite Score
percentile_columns = [f'{ratio} Percentile' for ratio in higher_is_better.keys()]
result_df['Composite Score'] = result_df[percentile_columns].mean(axis=1)

# Rank companies based on Composite Score
result_df['Rank'] = result_df['Composite Score'].rank(ascending=False).astype(int)

# Display results in ascending order of rank
result_df_sorted = result_df.sort_values(by='Rank')
result_df_sorted.reset_index(drop=True, inplace=True)
result_df_sorted

# Select top 90% based on rank
top_50_percent = result_df_sorted.head(int(len(result_df_sorted) * 0.9))
top_50 = top_50_percent['Company']
top_50

0          TCS.NS
1      DRREDDY.NS
2      HCLTECH.NS
3        WIPRO.NS
4         INFY.NS
5    NESTLEIND.NS
6          ABB.NS
7         ZEEL.NS
Name: Company, dtype: object

In [5]:
#Level 2: Growth
def fetch_growth_ratios(ticker):
    try:
        stock = yf.Ticker(ticker)
        
    
        # Fetching earningsGrowth
        earningsGrowth = pd.to_numeric(stock.info.get('earningsGrowth', 'N/A'), errors='coerce')
    
        # Fetching revenueGrowth
        revenueGrowth = pd.to_numeric(stock.info.get('revenueGrowth', 'N/A'), errors='coerce')
    
    
     
        return {
            'Company': ticker,
            'earningsGrowth': earningsGrowth,
            'revenueGrowth': revenueGrowth,
        }
    
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return {'Company': ticker}

# Fetch financial ratios for each company
growth_ratios = [fetch_growth_ratios(ticker) for ticker in top_50]

# Convert the list of dictionaries to a pandas DataFrame
result_df_growth = pd.DataFrame(growth_ratios)

# Drop rows with missing values
result_df_growth = result_df_growth.dropna()
result_df_growth



,Company,earningsGrowth,revenueGrowth
0,TCS.NS,0.022,0.040
1,DRREDDY.NS,0.103,0.066
2,HCLTECH.NS,0.056,0.053
3,WIPRO.NS,-0.074,-0.044
4,INFY.NS,-0.068,0.001
5,NESTLEIND.NS,0.071,0.083
6,ABB.NS,0.109,0.136
7,ZEEL.NS,0.085,0.202


In [6]:

higher_is_better = {
    'earningsGrowth': True,
    'revenueGrowth': True,
}

# Calculate percentiles for each ratio
for ratio, is_higher_better in higher_is_better.items():
    result_df_growth[f'{ratio} Percentile'] = result_df_growth[ratio].rank(pct=True, ascending=is_higher_better) * 100

# Calculate Composite Score
percentile_columns = [f'{ratio} Percentile' for ratio in higher_is_better.keys()]
result_df_growth['Composite Score'] = result_df_growth[percentile_columns].mean(axis=1)

# Rank companies based on Composite Score
result_df_growth['Rank'] = result_df_growth['Composite Score'].rank(ascending=False).astype(int)

# Display results in ascending order of rank
result_df_growth_sorted = result_df_growth.sort_values(by='Rank')
result_df_growth_sorted.reset_index(drop=True, inplace=True)

# Select top 90% based on rank
top_50_percent = result_df_growth_sorted.head(int(len(result_df_growth_sorted) * 0.9))
top_50 = top_50_percent['Company']
top_50

0          ABB.NS
1         ZEEL.NS
2      DRREDDY.NS
3    NESTLEIND.NS
4      HCLTECH.NS
5          TCS.NS
6         INFY.NS
Name: Company, dtype: object

In [7]:
#Level 3: Value
def fetch_value_ratios(ticker):
    try:
        stock = yf.Ticker(ticker)
        

         #Fetching enterpriseToRevenue
        enterpriseToRevenue = pd.to_numeric(stock.info.get('enterpriseToRevenue', 'N/A'), errors='coerce')
        
         #Fetching enterpriseToEbitda
        enterpriseToEbitda = pd.to_numeric(stock.info.get('enterpriseToEbitda', 'N/A'), errors='coerce')
    
         #Fetching trailingPE
        trailingPE = pd.to_numeric(stock.info.get('trailingPE', 'N/A'), errors='coerce')
    
        
    
     
        return {
            'Company': ticker,
            'enterpriseToRevenue': enterpriseToRevenue,
            'enterpriseToEbitda': enterpriseToEbitda,
            'trailingPE': trailingPE,
        }
    
    except Exception as e:
        print(f"Error fetching data for {ticker}: {e}")
        return {'Company': ticker}

# Fetch financial ratios for each company
value_ratios = [fetch_value_ratios(ticker) for ticker in top_50]

# Convert the list of dictionaries to a pandas DataFrame
result_df_value = pd.DataFrame(value_ratios)

# Drop rows with missing values
result_df_value = result_df_value.dropna()
result_df_value



,Company,enterpriseToRevenue,enterpriseToEbitda,trailingPE
0,ABB.NS,9.385,56.307,91.448490
1,ZEEL.NS,1.793,15.074,140.588230
2,DRREDDY.NS,3.703,12.405,20.078272
3,NESTLEIND.NS,12.779,54.187,82.863870
4,HCLTECH.NS,338.731,1640.946,28.706196
5,TCS.NS,5.902,23.004,32.856617
6,INFY.NS,369.295,1612.036,28.066679


In [8]:

higher_is_better = {
    'enterpriseToRevenue': False,
    'enterpriseToEbitda': False,
    'trailingPE': False,
}

# Calculate percentiles for each ratio
for ratio, is_higher_better in higher_is_better.items():
    result_df_value[f'{ratio} Percentile'] = result_df_value[ratio].rank(pct=True, ascending=is_higher_better) * 100

# Calculate Composite Score
percentile_columns = [f'{ratio} Percentile' for ratio in higher_is_better.keys()]
result_df_value['Composite Score'] = result_df_value[percentile_columns].mean(axis=1)

# Rank companies based on Composite Score
result_df_value['Rank'] = result_df_value['Composite Score'].rank(ascending=False).astype(int)

# Display results in ascending order of rank
result_df_value_sorted = result_df_value.sort_values(by='Rank')
result_df_value_sorted.reset_index(drop=True, inplace=True)

# Select top 90% based on rank
top_50_percent = result_df_value_sorted.head(int(len(result_df_value_sorted) * 0.9))
top_50 = top_50_percent['Company']
top_50

0      DRREDDY.NS
1         ZEEL.NS
2          TCS.NS
3    NESTLEIND.NS
4          ABB.NS
5         INFY.NS
Name: Company, dtype: object

In [ ]:
#with trading logic
from datetime import datetime, timedelta


def fetch_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

def preprocess_data(data):
    data["Return_6months"] = (data["Adj Close"] / data["Adj Close"].shift(126) - 1) * 100
    data["Return_3months"] = (data["Adj Close"] / data["Adj Close"].shift(63) - 1) * 100
    data["Return_1month"] = (data["Adj Close"] / data["Adj Close"].shift(21) - 1) * 100
    data["Return_1week"] = (data["Adj Close"] / data["Adj Close"].shift(5) - 1) * 100
    data["Return_1day"] = (data["Adj Close"] / data["Adj Close"].shift(1) - 1) * 100

    # Drop NaN values
    data = data.dropna()

    return data

nifty50_data = {}
start_date = "2018-07-01"
end_date = "2024-01-01"

for symbol in top_50:
    nifty50_data[symbol] = fetch_stock_data(symbol, start_date, end_date)

# Display trade data storage
trade_data = []

# Implement the trading strategy
for symbol, stock_data in nifty50_data.items():
    processed_data = preprocess_data(stock_data)

    # Initialize capital for each stock
    initial_capital = 100000
    capital = initial_capital

    # Track number of shares bought and selling state
    shares_held = 0
    in_trade = False

    # Track the start date of the trade
    trade_start_date = None

    # Iterate over the processed data
    for index, row in processed_data.iterrows():
        if (
            row["Return_6months"] > 0 and
            row["Return_3months"] > 0 and
            row["Return_1month"] > 0 and
            row["Return_1week"] > 0 and
            row["Return_1day"] > 0
        ):
            # Buy shares if not in a trade
            if not in_trade:
                shares_to_buy = int(capital / row["Close"])
                if shares_to_buy > 0:
                    capital -= shares_to_buy * row["Close"]
                    trade_data.append({"Symbol": symbol, "Action": "Buy", "Date": index, "Price": row["Close"], "Shares": shares_to_buy, "Capital": capital, "Profit/Loss": 0, "Days in Trade": 0})
                    shares_held = shares_to_buy
                    in_trade = True
                    trade_start_date = index

        elif in_trade and (row["Close"] - trade_data[-1]["Price"]) / trade_data[-1]["Price"] >= 0.2:
            # Sell shares if in a trade and there is a minimum increase of 5% or more
            capital += shares_held * row["Close"]
            profit_loss = (row["Close"] - trade_data[-1]["Price"]) * shares_held
            trade_data[-1]["Profit/Loss"] += profit_loss
            trade_data[-1]["Days in Trade"] = (index - trade_start_date).days
            trade_data.append({"Symbol": symbol, "Action": "Sell", "Date": index, "Price": row["Close"], "Shares": shares_held, "Capital": capital, "Profit/Loss": 0, "Days in Trade": 0})
            shares_held = 0
            in_trade = False

    # Sell remaining shares at the end
    if in_trade:
        capital += shares_held * processed_data["Close"].iloc[-1]
        profit_loss = (processed_data["Close"].iloc[-1] - trade_data[-1]["Price"]) * shares_held
        trade_data[-1]["Profit/Loss"] += profit_loss
        trade_data[-1]["Days in Trade"] = (processed_data.index[-1] - trade_start_date).days
        trade_data.append({"Symbol": symbol, "Action": "Sell", "Date": processed_data.index[-1], "Price": processed_data["Close"].iloc[-1], "Shares": shares_held, "Capital": capital, "Profit/Loss": 0, "Days in Trade": 0})

# Display trade data
trade_df = pd.DataFrame(trade_data)
print(trade_df)

# Write trade data to CSV
trade_df.to_csv("Funnel_QGV.csv", index=False)

# Display the final capital and percentage return for each stock
total_initial_capital = len(top_50) * initial_capital
total_final_capital = 0

for symbol in nifty50_data.keys():
    final_capital = initial_capital + trade_df[trade_df["Symbol"] == symbol]["Profit/Loss"].sum()
    total_final_capital += final_capital
    percentage_return = (final_capital - initial_capital) / initial_capital * 100
    print(f"Symbol: {symbol}, Initial Capital: {initial_capital:.2f}, Final Capital: {final_capital:.2f}, Returns: {percentage_return:.2f}%, Total Days in Trade: {trade_df[trade_df['Symbol'] == symbol]['Days in Trade'].sum()}")

# Define the symbol for Nifty50
nifty50_symbol = "^NSEI"

# Download Nifty50 historical data
nifty50_data = yf.download(nifty50_symbol, start=start_date, end=end_date)

# Calculate total return
start_value = nifty50_data["Adj Close"].iloc[0]
end_value = nifty50_data["Adj Close"].iloc[-1]
total_return = ((end_value / start_value) - 1) * 100

# Display overall returns
overall_returns = (total_final_capital - total_initial_capital) / total_initial_capital * 100
print("Total Initial Capital: ", total_initial_capital)
print("Total Final Capital Across All Stocks: ", total_final_capital)
print(f"Overall Returns Across All Stocks: {overall_returns:.2f}%")
# Display the total return of Nifty50
print(f"Total Return of Nifty50: {total_return:.2f}%")

# Calculate CAGR (Compound Annual Growth Rate)
start_year = int(start_date[:4])
end_year = int(end_date[:4])
years = end_year - start_year

# CAGR formula: ((final value / beginning value) ^ (1 / no of years)) - 1
cagr = ((total_final_capital / total_initial_capital) ** (1 / years)) - 1
print(f"CAGR (Compound Annual Growth Rate of Portfolio): {cagr:.2%}")

# Calculate and print Nifty CAGR
nifty_cagr = ((end_value / start_value) ** (1 / years)) - 1
print(f"CAGR (Compound Annual Growth Rate of Nifty): {nifty_cagr:.2%}")


In [11]:
#without trading logic
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices
from datetime import datetime, timedelta

def fetch_stock_data(symbol, start_date, end_date):
    stock_data = yf.download(symbol, start=start_date, end=end_date)
    return stock_data

def preprocess_data(data):
    data["Return_6months"] = (data["Adj Close"] / data["Adj Close"].shift(126) - 1) * 100
    data["Return_3months"] = (data["Adj Close"] / data["Adj Close"].shift(63) - 1) * 100
    data["Return_1month"] = (data["Adj Close"] / data["Adj Close"].shift(21) - 1) * 100
    data["Return_1week"] = (data["Adj Close"] / data["Adj Close"].shift(5) - 1) * 100
    data["Return_1day"] = (data["Adj Close"] / data["Adj Close"].shift(1) - 1) * 100

    

    return data

nifty50_data = {}
start_date = "2019-01-01"
end_date = "2024-01-01"

# Fetch Nifty50 data
for symbol in top_50:
    nifty50_data[symbol] = fetch_stock_data(symbol, start_date, end_date)

# trade data storage
trade_data = []

# Iterate over the stocks
for symbol, stock_data in nifty50_data.items():
    processed_data = preprocess_data(stock_data)

    # Check if there are enough data points
    if len(processed_data) < 2:
        print(f"Not enough data for {symbol}. Skipping...")
        continue

    # Buy shares on the start date
    initial_capital = 100000
    shares_bought = int(initial_capital / processed_data["Close"].iloc[0])
    trade_data.append({"Symbol": symbol, "Action": "Buy", "Buy Date": processed_data.index[0], "Price": processed_data["Close"].iloc[0], "Shares": shares_bought, "Capital": initial_capital, "Profit/Loss": 0, "Sell Date": None, "Days in Trade": 0})

    # Sell shares on the end date
    capital = shares_bought * processed_data["Close"].iloc[-1]
    profit_loss = capital - initial_capital
    trade_data[-1]["Sell Date"] = processed_data.index[-1]
    trade_data[-1]["Days in Trade"] = len(pd.bdate_range(trade_data[-1]["Buy Date"], trade_data[-1]["Sell Date"]))
    trade_data.append({"Symbol": symbol, "Action": "Sell", "Buy Date": None, "Sell Date": processed_data.index[-1], "Days in Trade": 0, "Price": processed_data["Close"].iloc[-1], "Shares": shares_bought, "Capital": capital, "Profit/Loss": profit_loss})

# Display trade data
trade_df = pd.DataFrame(trade_data)
print(trade_df)

trade_df.to_csv("top_50_All_Factors.csv", index=False)

# Calculate total initial capital and total final capital
total_initial_capital = trade_df[trade_df["Action"] == "Buy"]["Capital"].sum()
total_final_capital = total_initial_capital + trade_df[trade_df["Action"] == "Sell"]["Profit/Loss"].sum()

# Display the final capital, percentage return, and in-trade days for each stock
for symbol in nifty50_data.keys():
    final_capital = trade_df[(trade_df["Symbol"] == symbol) & (trade_df["Action"] == "Sell")]["Capital"].iloc[0]
    percentage_return = (final_capital - initial_capital) / initial_capital * 100
    in_trade_days = trade_df[(trade_df["Symbol"] == symbol) & (trade_df["Action"] == "Sell")]["Days in Trade"].iloc[0]
    print(f"Symbol: {symbol}, Final Capital: {final_capital:.2f}, Percentage Return: {percentage_return:.2f}%, In-Trade Days: {in_trade_days}")

# Define the symbol for Nifty50
nifty50_symbol = "^NSEI"

# Download Nifty50 historical data
nifty50_data = yf.download(nifty50_symbol, start=start_date, end=end_date)

# Calculate total return
start_value = nifty50_data["Adj Close"].iloc[0]
end_value = nifty50_data["Adj Close"].iloc[-1]
total_return = ((end_value / start_value) - 1) * 100

# Display the total percentage return across all stocks
total_percentage_return = (total_final_capital - total_initial_capital) / total_initial_capital * 100
print(f"Total Initial Capital: {total_initial_capital:.2f}")
print(f"Total Final Capital Across All Stocks: {total_final_capital:.2f}")
print(f"Total Percentage Return Across All Stocks: {total_percentage_return:.2f}%")
# Display the total return of Nifty50
print(f"Total Return of Nifty50: {total_return:.2f}%")

# trade data storage
y_trade_data = []
nifty_stocks = ["^NSEI"]    
# Iterate over the years
years = range(int(start_date[:4]), int(end_date[:4]))
for year in years:
    # Fetch stock data for each stock
    stock_data_dict = {}
    for symbol in top_50:
        stock_data_dict[symbol] = fetch_stock_data(symbol, f"{year}-01-01", f"{year}-12-31")

    #yearly returns
    yearly_returns = {}

    # Iterate over the stocks
    y_total_portfolio_value = 0
    y_total_initial_capital = 0
    for symbol, stock_data in stock_data_dict.items():
        # Check if there are enough data points
        if len(stock_data) < 2:
            print(f"Not enough data for {symbol} in {year}. Skipping...")
            continue

        # Buy shares on the start date
        y_initial_capital = 100000
        y_shares_bought = int(y_initial_capital / stock_data["Close"].iloc[0])
        y_capital = y_shares_bought * stock_data["Close"].iloc[-1]
        y_total_portfolio_value += y_capital
        y_total_initial_capital += y_initial_capital
   
    # Fetch stock data for each stock
    n_stock_data_dict = {}

    for symbol in nifty_stocks:
        n_stock_data_dict[symbol] = fetch_stock_data(symbol, f"{year}-01-01", f"{year}-12-31")

    # nifty yearly returns
    n_yearly_returns = {}

    # Iterate over the stocks
    n_total_portfolio_value = 0
    n_total_initial_capital = 0
    for symbol, stock_data in n_stock_data_dict.items():
        # Check if there are enough data points
        if len(stock_data) < 2:
            print(f"Not enough data for {symbol} in {year}. Skipping...")
            continue

        # Buy shares on the start date
        n_initial_capital = 100000
        n_shares_bought = int(n_initial_capital / stock_data["Close"].iloc[0])
        n_capital = n_shares_bought * stock_data["Close"].iloc[-1]
        n_total_portfolio_value += n_capital
        n_total_initial_capital += n_initial_capital

    # Append year and yearly return for the portfolio
    yearly_returns["Year"] = year
    yearly_returns["Nifty_50"] = ((n_total_portfolio_value - n_total_initial_capital) / n_total_initial_capital) * 100
    yearly_returns["Portfolio"] = ((y_total_portfolio_value - y_total_initial_capital) / y_total_initial_capital) * 100

    # Append yearly returns to trade data
    y_trade_data.append(yearly_returns)

#Display trade data
y_trade_df = pd.DataFrame(y_trade_data)
print(y_trade_df)


# Calculate CAGR (Compound Annual Growth Rate)
start_year = int(start_date[:4])
end_year = int(end_date[:4])
years = end_year - start_year

# CAGR formula: ((final value / beginning value) ^ (1 / no of years)) - 1
cagr = ((total_final_capital / total_initial_capital) ** (1 / years)) - 1
print(f"CAGR (Compound Annual Growth Rate of Portfolio): {cagr:.2%}")

# Calculate and print Nifty CAGR
nifty_cagr = ((end_value / start_value) ** (1 / years)) - 1
print(f"CAGR (Compound Annual Growth Rate of Nifty): {nifty_cagr:.2%}")

#Portfolio optimization
#Define the stock tickers and historical data
stock_tickers = top_50
start_year = int(start_date[:4])
end_year = int(end_date[:4])

# Function to fetch historical stock data
def fetch_historical_data(stock_ticker, start_date, end_date):
    return yf.download(stock_ticker, start=start_date, end=end_date)['Adj Close']

#Function to calculate portfolio returns for a given year
def calculate_portfolio_metrics(weights, year):
    #Fetch historical data for each stock
    stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{year}-01-01", f"{year+1}-01-01") for ticker in stock_tickers})

    #Calculate portfolio returns
    portfolio_returns = (stock_data.pct_change() * weights).sum(axis=1).dropna()

    #Calculate portfolio cumulative returns
    cumulative_returns = (portfolio_returns + 1).cumprod()

    #Calculate total return for the year
    total_return = cumulative_returns.iloc[-1] - 1

    return total_return, weights  #Return the total return and weights for the year

#Initialize empty dictionary to store annual returns and weights
annual_returns_and_weights = {}

#Loop through each year
for year in range(start_year, end_year):
    #Fetch historical stock data
    stock_data = pd.DataFrame({ticker: fetch_historical_data(ticker, f"{year}-01-01", f"{year+1}-01-01") for ticker in stock_tickers})

    #Calculate expected returns and sample covariance matrix of asset returns
    mu = expected_returns.mean_historical_return(stock_data)
    cov_matrix = risk_models.sample_cov(stock_data)

    #Optimize for maximum Sharpe ratio
    ef = EfficientFrontier(mu, cov_matrix)
    weights = ef.max_sharpe(risk_free_rate=-0.3)  #Assuming a risk-free rate of -3%

    #Calculate portfolio metrics for the year using the optimal weights
    annual_returns_and_weights[year] = calculate_portfolio_metrics(weights, year)

#Print annual returns and weights
for year, (return_value, weights) in annual_returns_and_weights.items():
    print(f"Year: {year}")
    print(f"Return: {return_value:.2%}")
    print("Weights:")
    for ticker, weight in weights.items():
        print(f"{ticker}: {weight:.2%}")
    print()


[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
          Symbol Action   Buy Date        Price  Shares        Capital  \
0     DRREDDY.NS    Buy 2019-01-01  2607.050049      38  100000.000000   
1     DRREDDY.NS   Sell        NaT  5797.899902      38  220320.196289   
2        ZEEL.NS    Buy 2019-01-01   480.250000     208  100000.000000   
3        ZEEL.NS   Sell        NaT   274.700012     208   57137.602539   
4         TCS.NS    Buy 2019-01-01  1902.800049      52  100000.000000   
5         TCS.NS   Sell        NaT  3793.399902      52  197256.794922   
6   NESTLEIND.NS    Buy 2019-01-01  1101.505005      90  100000.0000